# Imports

In [1]:
import os
from ultralytics import YOLO

# 1. YOLO v11 model

### 1.1 Training

In [2]:
configuration_file = os.path.join(os.path.dirname(os.getcwd()), 'data', 'dataset.yaml')


In [3]:
model_v11m = YOLO("yolo11l.pt")  # load the medium pretrained model

In [4]:
# Train the model
results = model_v11m.train(data=configuration_file, epochs=10, batch=16, device=0, pretrained=True, project='fine_tuning_v11l', name='epoch_10')

Ultralytics 8.3.56 🚀 Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolo11l.pt, data=/home/ec2-user/probe_detection/data/dataset.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=fine_tuning_v11l, name=epoch_10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_label

train: Scanning /home/ec2-user/probe_detection/data/yalo/train/labels.cache... 215 images, 0 backgrounds, 0 corrupt: 100%|██████████| 215/215 [00:00<?, ?it/s]
val: Scanning /home/ec2-user/probe_detection/data/yalo/val/labels.cache... 77 images, 0 backgrounds, 0 corrupt: 100%|██████████| 77/77 [00:00<?, ?it/s]


Plotting labels to fine_tuning_v11l/epoch_10/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 167 weight(decay=0.0), 174 weight(decay=0.0005), 173 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to fine_tuning_v11l/epoch_10
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10      10.8G      1.414      4.044      1.591          7        640: 100%|██████████| 14/14 [00:09<00:00,  1.41it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.42it/s]

                   all         77         77    0.00282      0.753    0.00517    0.00192


...

      10/10        11G      1.022      1.043      1.307          7        640: 100%|██████████| 14/14 [00:08<00:00,  1.59it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.47it/s]

                   all         77         77      0.938      0.784       0.88      0.673



10 epochs completed in 0.032 hours.
Optimizer stripped from fine_tuning_v11l/epoch_10/weights/last.pt, 51.2MB
Optimizer stripped from fine_tuning_v11l/epoch_10/weights/best.pt, 51.2MB

Validating fine_tuning_v11l/epoch_10/weights/best.pt...
Ultralytics 8.3.56 🚀 Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15102MiB)
YOLO11l summary (fused): 464 layers, 25,280,083 parameters, 0 gradients, 86.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.87it/s]


                   all         77         77      0.932      0.779       0.88      0.677
Speed: 0.2ms preprocess, 8.3ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to fine_tuning_v11l/epoch_10


### 1.2 Evaluation

In [5]:
metrics = model_v11m.val()

Ultralytics 8.3.56 🚀 Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15102MiB)
YOLO11m summary (fused): 303 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs


val: Scanning /home/ec2-user/probe_detection/data/yalo/val/labels.cache... 77 images, 0 backgrounds, 0 corrupt: 100%|██████████| 77/77 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<00:00,  2.92it/s]


                   all         77         77      0.695      0.679      0.744      0.584
Speed: 0.4ms preprocess, 14.0ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to fine_tuning_v11m/epoch_102


In [9]:
print(metrics.box.map)  # mAP50-95
print(metrics.box.map50)  # mAP50
print(metrics.box.map75)  # mAP75
print(metrics.box.maps)  # list of mAP50-95 for each category

0.5838085522661218
0.7439464398635709
0.6945134357018625
[    0.58381]


### 1.3 Testing on new Images